# **Step 1: Set Up Google Colab Environment**


To create a model for summarizing chats through fine-tuning, you'll need a step-by-step approach. We'll use a pre-trained model (like OpenAI's GPT or Hugging Face's transformers) and fine-tune it on your specific dataset to summarize large chats effectively. Here's a complete guide:

Step 1: Set Up Google Colab Environment
Open Google Colab and create a new notebook.
Ensure you have a GPU runtime enabled (Runtime > Change runtime type > Hardware Accelerator > GPU).

# **Step 2: Install Required Libraries**

In [ ]:
!pip install transformers datasets accelerate
!pip install wandb  # Optional, for experiment tracking


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# **Step 3: Prepare Your Dataset**

In [ ]:
import csv
import random

# Generate a large dataset with dialogue and summary pairs
def generate_large_dataset(num_samples=1000):
    data = []
    for i in range(num_samples):
        dialogue = f"User: Hello, this is message {i}. How are you?\nAssistant: I'm fine, thank you. How can I assist you with message {i}?"
        summary = f"User greeted and inquired about assistance for message {i}."
        data.append({"dialogue": dialogue, "summary": summary})
    return data

# Create the dataset
data = generate_large_dataset(1000)

# Write the dataset to a CSV file
csv_file_path = "large_dialogue_summary_dataset.csv"

with open(csv_file_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["dialogue", "summary"])
    writer.writeheader()
    writer.writerows(data)

print(f"CSV file has been created: {csv_file_path}")


CSV file has been created: large_dialogue_summary_dataset.csv


In [ ]:
from datasets import load_dataset

# Replace 'your_dataset_path.csv' with your uploaded dataset path
dataset = load_dataset('csv', data_files='large_dialogue_summary_dataset.csv')
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['dialogue', 'summary'],
        num_rows: 1000
    })
})


In [ ]:
# Split into train and validation datasets
train_dataset = dataset['train'].train_test_split(test_size=0.1)['train']
val_dataset = dataset['train'].train_test_split(test_size=0.1)['test']


# **Step 4: Choose a Pre-trained Model**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

# **Step 5: Preprocess the Data**

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(examples['dialogue'], max_length=1024, truncation=True, padding='max_length')
    outputs = tokenizer(examples['summary'], max_length=128, truncation=True, padding='max_length')
    inputs['labels'] = outputs['input_ids']
    return inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

# **Step 6: Define the Data Collator**

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


# **Step 7: Set Up Training Arguments**

In [ ]:
import os
from transformers import Seq2SeqTrainingArguments

# Disable Weights & Biases (wandb)
os.environ["WANDB_DISABLED"] = "true"

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    fp16=True,  # Use mixed precision for faster training on GPU
    logging_dir='./logs',
    logging_steps=10,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# **Step 8: Train the Model**

In [ ]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

# Define Data Collator for padding and batch consistency
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,  # Replace with your tokenized training dataset
    eval_dataset=tokenized_val,     # Replace with your tokenized validation dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()

# Save the final model
trainer.save_model("./final_chat_summary_model")
tokenizer.save_pretrained("./final_chat_summary_model")

print("Training completed! The fine-tuned model is saved to './final_chat_summary_model'.")


<ipython-input-13-790e0e360ec1>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,0.003800,0.040804
2,0.002000,0.025514
3,0.000900,0.034156


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Training completed! The fine-tuned model is saved to './final_chat_summary_model'.


# **Step 9: Evaluate the Model**

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
# Install the required dependency
!pip install rouge_score

# Now import and use the ROUGE metric
import evaluate
from datasets import load_dataset

# Load ROUGE metric
rouge = evaluate.load("rouge")

# Function to compute metrics during evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = tokenizer.batch_decode(logits, skip_special_tokens=True)
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE score
    results = rouge.compute(predictions=predictions, references=references)
    return results


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=73dafde7b7d5fafaf75f0d8f4d756cdb17ccbf5c76533b752ee41a9806c119da
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


# **Step 10: Test the Model**

In [ ]:
# Import torch
import torch

# Make sure to move the model and input tensors to the same device (GPU if available, else CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Move model to the correct device
model = model.to(device)

# Tokenize the input chat and move inputs to the correct device
inputs = tokenizer(chat, return_tensors="pt", max_length=1024, truncation=True).to(device)

# Generate the summary
summary_ids = model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)

# Decode and print the summary
print("Summary:", tokenizer.decode(summary_ids[0], skip_special_tokens=True))


Summary: User greeted and inquired about assistance for 12345 12345.User asked for assistance for assistance with order 12345 and asked for help canceling.User canceled the order and thanked the user for assistance.User and assistance were greeted and greeted by and assistance for the 12345, and asked about help for other orders.
